In [17]:
from cvxopt import matrix, solvers
from Utils.signals import *

In [18]:
# Let's assume we are interested in the following stocks: AAPL, MSFT, TSLA, AMZN, GOOG
tickers = ['AAPL', 'MSFT', 'TSLA', 'AMZN', 'GOOG']
dates = pd.date_range(start='2020-01-14', periods=3)

# Download historical stock data
data = yf.download(tickers, start='2020-01-14', end='2020-01-17')
display(data)

[*********************100%***********************]  5 of 5 completed


Price           Close                                                    High  \
Ticker           AAPL       AMZN       GOOG        MSFT       TSLA       AAPL   
Date                                                                            
2020-01-14  75.602058  93.472000  71.206734  154.481079  35.861332  76.784402   
2020-01-15  75.278084  93.100998  71.620773  155.481506  34.566666  76.283920   
2020-01-16  76.221046  93.897003  72.242821  158.330475  34.232666  76.332274   

Price                                                    ...       Open  \
Ticker           AMZN       GOOG        MSFT       TSLA  ...       AAPL   
Date                                                     ...              
2020-01-14  94.355499  71.750157  155.881729  36.493999  ...  76.574048   
2020-01-15  93.943001  71.730005  156.205660  35.855999  ...  75.401398   
2020-01-16  94.279503  72.257256  158.397179  34.297333  ...  75.822099   

Price                                                       Volume            \
Ticker           AMZN       GOOG        MSFT       TSLA       AAPL      AMZN   
Date                                                                           
2020-01-14  94.293999  71.611319  155.681630  36.284000  161954400  68818000   
2020-01-15  93.612503  71.173391  154.947928  35.317333  121923600  57932000   
2020-01-16  94.149498  72.030827  156.596346  32.916668  108829200  53190000   

Price                                      
Ticker          GOOG      MSFT       TSLA  
Date                                       
2020-01-14  31178000  23477400  434943000  
2020-01-15  25654000  21417900  260532000  
2020-01-16  23474000  23865400  326050500  

[3 rows x 25 columns]

In [19]:
# Compute historical daily returns
var_returns = data['Close'].pct_change().dropna()
display(var_returns)

Ticker,AAPL,AMZN,GOOG,MSFT,TSLA
Date,,,,,
2020-01-15,-0.004285,-0.003969,0.005815,0.006476,-0.036102
2020-01-16,0.012526,0.008550,0.008685,0.018324,-0.009662


In [4]:
# Expected returns (mean of past returns)
mu = var_returns.mean().values.reshape(-1, 1)
print(mu)

[[ 0.00412056]
 [ 0.00229039]
 [ 0.00724995]
 [ 0.01239979]
 [-0.02288225]]


In [5]:
# Compute covariance matrix
Sigma = var_returns.cov().values
print(Sigma)

[[1.41315820e-04 1.05232851e-04 2.41304377e-05 9.95877686e-05
  2.22246137e-04]
 [1.05232851e-04 7.83631509e-05 1.79690763e-05 7.41594595e-05
  1.65498772e-04]
 [2.41304377e-05 1.79690763e-05 4.12040226e-06 1.70051481e-05
  3.79497254e-05]
 [9.95877686e-05 7.41594595e-05 1.70051481e-05 7.01812697e-05
  1.56620801e-04]
 [2.22246137e-04 1.65498772e-04 3.79497254e-05 1.56620801e-04
  3.49524529e-04]]


In [6]:
alpha_scores = [ 0.5, 0.5, 0.5, 0.5, 0.5 ]
print(alpha_scores)

[0.5, 0.5, 0.5, 0.5, 0.5]


In [7]:
n_assets = 5

# Quadratic term (Risk component: λ * w'Σw)
P = matrix(0.3 * Sigma)

# Normalize signal scores
normalized_scores = alpha_scores / np.sum(alpha_scores)

# Linear term (maximize return + signal influence)
q = matrix(-mu - normalized_scores.reshape(-1, 1))
print(q)

[-2.04e-01]
[-2.02e-01]
[-2.07e-01]
[-2.12e-01]
[-1.77e-01]



In [8]:
# Constraints
G = -np.eye(n_assets)   # Only non-negative weights constraint
h = np.zeros(n_assets)  # w ≥ 0

# Full investment constraint: sum(w) = 1
A = matrix(np.ones((1, n_assets)))
b = matrix(np.ones(1))

# Solve quadratic optimization problem
sol = solvers.qp(P, q, G=matrix(G), h=matrix(h), A=A, b=b)

# Extract portfolio weights
weights = np.array(sol['x']).flatten()

     pcost       dcost       gap    pres   dres
 0: -2.0137e-01 -1.2124e+00  1e+00  0e+00  3e+00
 1: -2.0164e-01 -2.2239e-01  2e-02  1e-16  6e-02
 2: -2.0717e-01 -2.1249e-01  5e-03  7e-17  1e-02
 3: -2.1207e-01 -2.1285e-01  8e-04  3e-15  3e-17
 4: -2.1239e-01 -2.1239e-01  8e-06  2e-16  4e-17
 5: -2.1239e-01 -2.1239e-01  8e-08  1e-16  4e-17
Optimal solution found.
